In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']

In [2]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

station_list = ["SS20", "Expert", "Our"]

measures = pd.read_pickle(f'data/ss20_cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/ss20_our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["stature"])
measures = measures[measures['measures_station'].isin(["SS20", "Expert measurer 1", "our"])]
measures.replace("Expert measurer 1", "Expert", inplace=True)
measures.replace("our", "Our", inplace=True)
measures.reset_index(drop=True, inplace=True)

In [3]:
#measures[:116].subject[98], measures[:116].subject[99]

('IEEEP1_47', 'IEEEP1_47')

In [4]:
#measures.iloc[98], measures.iloc[99]

(bust_chest_girth     1007.89093
 waist_girth          851.851501
 hip_girth           1477.041748
 thigh_girth            886.5271
 upper_arm_girth      301.723999
 neck_girth           391.542938
 neck_waist           437.212769
 repetition                    1
 gender                     male
 measures_station            Our
 subject               IEEEP1_47
 Name: 98, dtype: object,
 bust_chest_girth    984.115967
 waist_girth         845.393921
 hip_girth           961.967285
 thigh_girth         542.337524
 upper_arm_girth     298.693359
 neck_girth          366.089539
 neck_waist          424.065216
 repetition                   2
 gender                    male
 measures_station           Our
 subject              IEEEP1_47
 Name: 99, dtype: object)

In [29]:
numeric = ['upper_arm_girth', 'neck_waist']
np.set_printoptions(suppress=True)
data = measures[:118][measures[:118]['repetition'] == 1][numeric].to_numpy() - measures[:118][measures[:118]['repetition'] == 2][numeric].to_numpy()
#data

array([[  1.40893555,   4.64007568],
       [ -6.6887207 ,  -2.17272949],
       [ -1.2623291 ,  -5.51266479],
       [  3.1187439 ,   3.49002075],
       [ -0.19030762,   4.11608887],
       [ 14.13397217,  -5.46859741],
       [ 10.21157837,  -1.03594971],
       [ -0.22207642,   2.34451294],
       [ 15.456604  ,   2.98956299],
       [ -0.22415161,  13.8132019 ],
       [ -0.66098022,  14.04638672],
       [ -1.06271362,  -5.71899414],
       [ -2.43322754, -10.23236084],
       [-15.46234131,  -6.95504761],
       [ -8.52267456,   0.44985962],
       [-20.50009155,  -9.95068359],
       [  8.18029785,  -2.85958862],
       [ -6.08718872,   5.1171875 ],
       [ 11.04663086,  10.12286377],
       [ -6.57989502,   1.82684326],
       [ -3.08831787,   7.29937744],
       [  0.15945435,  -7.35684204],
       [  1.53921509,  72.23977661],
       [ -1.0864563 ,  -3.71673584],
       [  6.57867432,  -1.13223267],
       [ -9.97293091, -25.0333252 ],
       [  9.94958496,  -6.49005127],
 

In [6]:
measures.drop([98,99], inplace=True)

# CALC ICC

In [7]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure)['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [8]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.879662,15.619881,57,58,1.270953e-20,"[0.81, 0.93]"
1,ICC2,Single random raters,0.879738,15.785392,57,57,1.827924e-20,"[0.81, 0.93]"
2,ICC3,Single fixed raters,0.880849,15.785392,57,57,1.827924e-20,"[0.81, 0.93]"
3,ICC1k,Average raters absolute,0.935979,15.619881,57,58,1.270953e-20,"[0.89, 0.96]"
4,ICC2k,Average random raters,0.936022,15.785392,57,57,1.827924e-20,"[0.89, 0.96]"
5,ICC3k,Average fixed raters,0.936650,15.785392,57,57,1.827924e-20,"[0.89, 0.96]"


In [9]:
our_icc.index = ['Expert', 'Our', "SS20"]
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist
Expert,0.992658,0.983699,0.981579,0.976319,0.986107,0.980247,0.870923
Our,0.997703,0.996359,0.995180,0.992597,0.973305,0.992799,0.880849
SS20,0.997617,0.994610,0.993753,0.981880,0.990776,0.992513,0.965700


# CALC SEM

In [10]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [11]:
our_sem.index = ['Expert', 'Our', "SS20"]
our_sem.T.round(4)

,Expert,Our,SS20
bust_chest_girth,10.9598,6.0415,6.6435
waist_girth,17.3445,8.9285,10.3207
hip_girth,15.1953,6.7070,7.8820
thigh_girth,11.0017,5.7375,9.1193
upper_arm_girth,5.0882,7.2454,4.4127
neck_girth,5.9301,4.3116,4.3156
neck_waist,10.6182,11.6429,7.5959


# CALC PSD

In [12]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [13]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"SS20/Expert": neck_result.loc["SS20"]["Expert"]})
neck_result = neck_result.loc['Our'].drop('Our').append(MX)

In [14]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"SS20/Expert": bust_result.loc["SS20"]["Expert"]})
bust_result = bust_result.loc['Our'].drop('Our').append(MX)

In [15]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"SS20/Expert": waist_result.loc["SS20"]["Expert"]})
waist_result = waist_result.loc['Our'].drop('Our').append(MX)

In [16]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"SS20/Expert": hip_result.loc["SS20"]["Expert"]})
hip_result = hip_result.loc['Our'].drop('Our').append(MX)

In [17]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"SS20/Expert": arm_result.loc["SS20"]["Expert"]})
arm_result = arm_result.loc['Our'].drop('Our').append(MX)

In [18]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"SS20/Expert": thigh_result.loc["SS20"]["Expert"]})
thigh_result = thigh_result.loc['Our'].drop('Our').append(MX)

In [19]:
neck_waist_result = calculate_psd(measure='neck_waist')
MX = pd.Series({"SS20/Expert": neck_waist_result.loc["SS20"]["Expert"]})
neck_waist_result = neck_waist_result.loc['Our'].drop('Our').append(MX)

In [30]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2

        
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)

        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['Our','Expert','SS20']).T

In [31]:
calculate_mad()

,Our,Expert,SS20
bust_chest_girth,7.630763,10.181818,7.296091
waist_girth,7.901983,16.227273,9.793399
hip_girth,7.131373,13.500000,7.163131
thigh_girth,6.089542,12.590909,5.850343
upper_arm_girth,4.507424,5.000000,7.947104
neck_girth,4.574153,6.545455,5.226101
neck_waist,7.479008,11.636364,10.034299


In [22]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "125%"),
                              ("color", 'black'),
                              ("font-weight", '700')])]

In [23]:
mad_table = calculate_mad().round(4)
mad_table = mad_table.style.set_caption("OUR vs SS20 MAD statistic result").set_table_styles(styles)
mad_table.format(precision=4)

,SS20,Expert,Our
bust_chest_girth,7.6308,10.1818,7.2961
waist_girth,7.9020,16.2273,9.7934
hip_girth,7.1314,13.5000,7.1631
thigh_girth,6.0895,12.5909,5.8503
upper_arm_girth,4.5074,5.0000,7.9471
neck_girth,4.5742,6.5455,5.2261
neck_waist,7.4790,11.6364,10.0343


In [24]:
sem_table = our_sem.T.round(4)
sem_table = sem_table.style.set_caption("OUR vs SS20 SEM statistic result").set_table_styles(styles)
sem_table.format(precision=4)

,Expert,Our,SS20
bust_chest_girth,10.9598,6.0415,6.6435
waist_girth,17.3445,8.9285,10.3207
hip_girth,15.1953,6.7070,7.8820
thigh_girth,11.0017,5.7375,9.1193
upper_arm_girth,5.0882,7.2454,4.4127
neck_girth,5.9301,4.3116,4.3156
neck_waist,10.6182,11.6429,7.5959


In [25]:
data_psd = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result, neck_waist_result],
    index=our_semantic[3:-1]).round(2)
data_psd.columns = ['Our vs Expert', "Our vs SS20", 'SS20 vs Expert']
data_psd.to_csv("output/our_psd.csv")
data_psd = data_psd.style.set_caption("OUR vs SS20 PSD statistic result").set_table_styles(styles)
data_psd.format(precision=2)

,Our vs Expert,Our vs SS20,SS20 vs Expert
bust_chest_girth,24.75,16.16,23.64
waist_girth,32.63,22.11,36.19
hip_girth,63.70,38.00,54.94
thigh_girth,34.13,24.55,32.24
upper_arm_girth,14.11,32.56,33.07
neck_girth,24.05,17.77,22.63
neck_waist,32.11,31.82,32.56
